Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices, seed, choices

import lab9_lib
import numpy as np
import copy

In [2]:
fitness = lab9_lib.make_problem(2)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11011011010101010010101000100110011100101010001111: 25.60%
00111001111001000001000100000001101010001000100010: 18.60%
10110000110000110000110001111011010110001110000100: 44.00%
00111100000000010100011111111101010111000000011101: 30.40%
10010110100010111000001011011011001100101001000101: 24.00%
10111111001110011111001100011111011000001111011111: 30.80%
10100001010100000101001000001010010110100011111101: 20.00%
10000101010001010111001101011011111001010100111011: 32.00%
10011011110000001100000010101000011000110010001000: 22.80%
10011111001101000111001101110010101000110110111011: 27.20%
10


# Ideas

## Tricks and ideas

- Save configurations + fitness in a dictionary to avoid repetitions
- Create a CNN+FC to, first extract the genome information (necessary spatial information) and then predict the associated fitness


## Island

- Generate K islands, evolve them independently and, each N mutations, let one individual from each travel to others (one individual can travel to multiple islands?). How to choose the individual?


## Mutation

- Change the bit with a given probability. How many bits to mutate?


## Save the best

- Elitism: copy the champion into the offsprings without modifications
- Valhalla: keep the best apart and, from time to time, put them back in the population


## Recombination

- Implement one cuts/two cuts/n cuts
- Do it choosing the idx through a Gaussian


## Parent selection (semi stochastic)

- Try roulette (linearized proportional to fitness, trouble with big populations)
- Try tournament (increase size to increase pressure)


## Survival selection (fully deterministic)

- Filter only the top K fittest individuals

## Tricks and ideas

- Save configurations + fitness in a dictionary to avoid repetitions
- Create a CNN+FC to, first extract the genome information (necessary spatial information) and then predict the associated fitness


## Island

- Generate K islands, evolve them independently and, each N mutations, let one individual from each travel to others (one individual can travel to multiple islands?). How to choose the individual?


## Mutation

- Change the bit with a given probability. How many bits to mutate?


## Save the best

- Elitism: copy the champion into the offsprings without modifications
- Valhalla: keep the best apart and, from time to time, put them back in the population


## Recombination

- Implement one cuts/two cuts/n cuts


## Parent selection (semi stochastic)

- Try roulette (linearized proportional to fitness, trouble with big populations)
- Try tournament (increase size to increase pressure)


## Survival selection (fully deterministic)

- Filter only the top K fittest individuals

# Our Solution

In [17]:
fitness = lab9_lib.make_problem(2)
seed(42)

for n in range(10):
    ind = choices([0, 1], k=10)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

1000111000: 29.00%
0100110110: 28.00%
1100100011: 28.00%
1111011111: 46.00%
1000000010: 20.00%
0001110100: 29.00%
1111110000: 60.00%
0110101000: 29.00%
1011001100: 28.00%
0110001111: 60.00%
10


## V1.0

In [18]:
def mutate_bit(individual : list) -> list:
    
    idx = np.random.randint(0, len(individual))
    mutated_individual = copy.deepcopy(individual)
    mutated_individual[idx] = int(not(bool(mutated_individual[idx])))
    return mutated_individual


def roulette_selection(top_k : int, individuals : list, fitness : np.ndarray) -> list:

    # Select top K fittest individuals and return N_OFFSPRINGS couple of individuals.
    parents = choices(individuals, weights=np.array(fitness)/sum(fitness), k=top_k)
    return choices([(parent1, parent2) for i, parent1 in enumerate(parents) for j, parent2 in enumerate(parents) if i != j], k=N_OFFSPRINGS)


def one_cut_xover(couple : tuple) -> list:

    idx = np.random.randint(1, len(couple[0]) - 1)
    return couple[0][:idx] + couple[1][idx:]

In [20]:
POP_SIZE = 100          # Number of parents
N_OFFSPRINGS = 300      # Number of children
N_GENERATIONS = 10000   # Number of generations
TOP_K = 30              # Number of parents from which generating the offsprings

italy = []
for _ in range(POP_SIZE):
    italy.append(choices([0, 1], k=30))
        

In [23]:
parents = roulette_selection(TOP_K, italy, [fitness(ind) for ind in italy])

for _ in range(N_GENERATIONS):
    
    recombinated_offsprings = map(one_cut_xover, parents)
    mutated_offsprings = np.array(list(map(mutate_bit, recombinated_offsprings)))
    fit = np.array(list(map(fitness, mutated_offsprings)))
    survived = mutated_offsprings[np.argsort(fit) < POP_SIZE]
    if (fit > 0.85).any():
        break
    print(_)

0
1
2
3
4
5
6
7
8
9
10
11


In [46]:
survived_fitness = list(map(fitness, survived))
ordered_survived = [_ for _, x in sorted(zip(survived, survived_fitness), key=lambda pair: pair[1], reverse=True)]

for sur in ordered_survived:
    print(f"{''.join(str(g) for g in sur)}: {fitness(sur):.2%}")

111001110111111000111111111111: 80.00%
010010001110101111111101011001: 60.00%
001011110111101111001001100101: 60.00%
000010111111100111111111000100: 60.00%
101001010011101110011111110100: 60.00%
011001111111001001010110101011: 60.00%
001001101101100111110100001011: 53.33%
100010111100010101101111001100: 53.33%
001011110101101001110010000111: 53.33%
100010111101110110110101100000: 53.33%
001001011001111110000001011010: 46.67%
111110001001000100110011100001: 46.67%
100010111011111010101011011010: 41.67%
011110001001000010110100000011: 40.00%
111111110101111000111111111111: 39.33%
110100010001010101111111011100: 38.33%
111100011010101010101011011110: 38.00%
110100010001010101111111111011: 37.67%
111111011101001000010111011111: 37.33%
000010111110100111111111111010: 37.33%
011001111111011001111111111100: 36.67%
001011110011111000111111111111: 36.67%
111100111111100101111111110100: 36.67%
110100010001010101111111001100: 35.00%
101011111110100010111100000011: 34.67%
1000100011101011111111000

## V2.0

In [ ]:
N_ISLANDS = 5           # Number of different populations
POP_SIZE = 100          # Number of parents
N_OFFSPRINGS = 300      # Number of children
N_GENERATIONS = 10000   # Number of generations
TOP_K = 30              # Number of parents from which generating the offsprings


globe = {f'island{k}' : [] for k in range(N_ISLANDS)}

for n in range(N_ISLANDS):
    for _ in range(POP_SIZE):
        globe[f'island{n}'].append(choices([0, 1], k=30))
        
italy = globe['island0']

In [ ]:
def mutate_bits(individual : list) -> list:
    '''
    Mutate every bit with a given probability
    '''
    # TODO
    idx = np.random.randint(0, len(individual), size = len(individual))
    mutated_individual = copy.deepcopy(individual)
    np.random.random(size = len(individual))
    if np.random.random() < 0.9:
        mutated_individual[idx] = int(not(bool(mutated_individual[idx])))
    return mutated_individual
